In [5]:
import os

modelname = 'src/model1.1.1_pan/'
os.system('mkdir -p ./'+modelname)
# 创建显示数据信息文件outputf
outputf = open(modelname+'/output.txt', 'w')
outputf.write('OMP_NUM_THREADS = '+str(os.popen('echo $OMP_NUM_THREADS').read()) +'\n\n')

21

In [6]:
import numpy as np
from numpy import *
import matplotlib.pyplot as plt
import keras, os, struct

In [7]:
#lsstr = "ls /home/xiaodongli/data/colas/cola_multiverse/om_As/"
lsstr = "ls /media/cosmo/Seagate/cola_multiverse/om_As/"

# 本代码内无用
def cosmostr(om, As):
    return 'om%.3f' % om + '_As%.3f' % As

# 本代码内无用
def snpfiles(cosmology, snpstr='c'):  #
    return os.popen(lsstr + cosmology + "*snap*" + snpstr + ".*").read().split()

# 打开含grid内容的文件
def gridfiles(cosmology, snpstr='c'):
    return os.popen(lsstr + cosmology + "*grid*" + snpstr + ".*").read().split()

# 返回[sigma8, r m]
def mocklist():
    # *代替多个字母,列出符合的.lua文件（列表类型，因数目非一个）,lua文件包含宇宙学参数信息等
    files = os.popen(lsstr + "om*.lua").read().split('\n')  # 列表类型
    cosmologies = []  # 宇宙学样本名称列表
    mocks = {}  # 模拟的数据参数
    ifile = 0  # 有效文件
    for nowfile in files:
        nowstr = nowfile[-39:-10]
        # eg. om0.160_As2.000_sigma8_0.4252_init
        # str[a:b]不存在时,返回'',不存在则忽略
        if nowstr == '':
            continue
        cosmologies.append(nowstr[0:15])  #添加宇宙学标签：om0.160_As2.000
        ifile += 1
        try:
            mocks[nowstr[0:15]] = {'om': float(nowstr[2:7]), 'As': float(nowstr[10:15]),
                                   'sigma8': float(nowstr[23:29])}
            # 添加ranseed标签
            ranseed = float(open(nowfile, 'r').readline().split()[2])
            mocks[nowstr[0:15]]['ranseed'] = int(ranseed)
            # print(ranseed)
        except:
            pass
    return cosmologies, files, mocks  # 名称列表， 样本文件， 参数列表
# grid样本集合
gridfile_dict = {}
# 提取样本数据
cosmologies, filenames, infos = mocklist()
print('In total ', len(cosmologies), 'cosmologies')
outputf.write('In total '+str(len(cosmologies))+'cosmologies\n')
print('Build up gridfile_dict... (for speed-up of load_grid()) ')
outputf.write('Build up gridfile_dict... (for speed-up of load_grid()) \n')
for cosmology in cosmologies:
    rlt = gridfiles(cosmology)  # 打开标签相应的grid文件(列表类型）
    if rlt == []:
        print ('\tmissing cosmology!', cosmology)
        outputf.write('\tmissing cosmology!' +str(cosmology)+ '\n')
    else:
        gridfile_dict[cosmology] = rlt[0]
np.random.shuffle(cosmologies)  #

In total  465 cosmologies
Build up gridfile_dict... (for speed-up of load_grid()) 
	missing cosmology! om0.180_As2.040
	missing cosmology! om0.180_As2.100
	missing cosmology! om0.180_As2.120
	missing cosmology! om0.180_As2.140
	missing cosmology! om0.180_As2.200
	missing cosmology! om0.180_As2.220
	missing cosmology! om0.180_As2.260


In [8]:
def load_grid(gridfile, snpstr='c', printinfo=False):  # 网格加载
    #gridfile = os.popen(lsstr + cosmology+"_sigma8_*grid*" + snpstr + ".*").read().split()[0]
    #print('load in gridfile : ', gridfile, '...')
    nowf = open(gridfile, 'rb')  # 以二进制形式读取文件
    # struct:对python基本类型值与用python字符串格式表示的C struct类型间转化
    size = struct.unpack('f' * 1, nowf.read(4 * 1))[0]
    grid_nc = struct.unpack('i' * 1, nowf.read(4 * 1))[0]
    data = struct.unpack('f' * grid_nc ** 3, nowf.read(4 * grid_nc ** 3))
    if printinfo:
        print('read in box size     \n\t', size)
        print('read in num_grid      \n\t', grid_nc)
        print('read in coarse grid \n\tsize    : ', len(data), '\n\texpect  : ', grid_nc ** 3)

    nowf.close()
    return np.array(data).reshape((grid_nc, grid_nc, grid_nc))

def subcubes(A):
    rlt = []
    for row1 in [0, 32, 64, 96]:
        for row2 in [0, 32, 64, 96]:
            for row3 in [0, 32, 64, 96]:
                rlt.append(A[row1:row1+32,row2:row2+32,row3:row3+32])
    return rlt

def data_augument(A):
    rlt = []

In [13]:
test_size = 0.3
batch_size = 6
num_subcube = 64
num_data_augument = 48

x_test = np.zeros((int(test_size*batch_size)+1,32,32,32, 1))
y_test = np.zeros((int(test_size*batch_size)+1, 2))


###  xiaodong: 重新写了 load_grid 程序。。。之前有错误！！！！（好像只会 load 进来一个 om...)
def train_generator():  # 必须无限循环yield数据,全部数据遍历后再重新遍历数据,为下一个epoch yield 数据
    i = 0
    while 1:
        X = []
        y = []
        global x_test, y_test,test_size, batch_size
        for cosmology in cosmologies[batch_size * i: batch_size * (i + 1)]:
            try:
                gridfile = gridfile_dict[cosmology]
                griddata = load_grid(gridfile, 'c')
                for subcube in subcubes(griddata):
                    X.append(subcube)
                    y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
            except KeyError:
                pass
                
        X = np.array(X
        y = np.array(y)
        x_train, x_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=test_size)
        x_train = x_train.reshape(-1, 32, 32, 32, 1)
        x_test = x_test.reshape(-1, 32, 32, 32, 1)
        i += 1
        yield x_train, y_train  # tuple 类型
        # 15个批次后重新遍历数据,此循环即死循环
        if i == 465//batch_size:
            i = 0


# In[46]:


from keras import Sequential, layers
from sklearn import model_selection, metrics


def create_validate_sample(nsample, use_random=True, startid=None ):
    cosmologies = list(gridfile_dict); ncosmo = len(cosmologies)
    if use_random:
        rows = [np.random.randint(0,ncosmo) for row in range(nsample)];
        rows = list(set(rows))
        while len(rows) < nsample:
            rows = rows + [np.random.randint(0,ncosmo) for row in range(nsample - len(rows))];
            rows = list(set(rows))
    else:
        rows = range(startid, startid+nsample)
    x, y =[], []
    for row in rows:
        cosmology = cosmologies[row]
        gridfile = gridfile_dict[cosmology]
        griddata = load_grid(gridfile)
        for subcube in subcubes(griddata):
            x.append(subcube)
            y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
    x = np.array(x); x = x.reshape(-1, 32, 32, 32, 1); y = np.array(y)
    return x, y

def plot_test(model, x, y, plot_avg_predict = True, fig=None, ax = None, plot_subpoints=False):
    y_predict = model.predict(x); 
    
    if fig == None or ax ==None:
        fig, ax = plt.subplots(figsize=(14,6))
    cs = range(len(y)); cs = cs / mean(cs)
    ax.scatter(y[:,0], y[:,1], c='b', marker='*', label='input', s=200)
    if plot_subpoints:
        ax.scatter(y_predict[:,0], y_predict[:,1], c='g',  marker='p', s=50, label='outputs')
    
        
    om_test, w_test = y[:,0], y[:,1]
    om_predict, w_predict = y_predict[:,0], y_predict[:,1]
    
    if plot_avg_predict:
        ax.scatter(mean(y_predict[:,0]), mean(y_predict[:,1]), marker='*', c='r', label='output_avg', s=200)
        ax.plot([om_test[0], mean(y_predict[:,0])], [w_test[0], mean(y_predict[:,1])], lw=2, c='k', ls='--' )

    
    #for row in range(len(om_predict)):
    #    ax.plot( [om_predict[row], om_test[row]], [w_predict[row], w_test[row]], lw=0.5, c='gray' )
    ax.set_xlabel(r'$\Omega_m$',fontsize=16); ax.set_ylabel(r'$\sigma_8$',fontsize=16)
    ax.legend()   
    return fig, ax

In [10]:
max_epochs = 300

nowmodel = keras.Sequential([
        layers.BatchNormalization( input_shape=(32, 32, 32, 1)),
        layers.Conv3D(32, (3, 3, 3), activation='relu'),
        layers.AveragePooling3D(pool_size=(2, 2, 2)),
        layers.BatchNormalization(),
        layers.Conv3D(64, (3, 3, 3), activation='relu'),
        layers.AveragePooling3D(pool_size=(2, 2, 2)),
        layers.BatchNormalization(),
        layers.Conv3D(128, (3, 3, 3), activation='relu'),
        layers.AveragePooling3D(pool_size=(2, 2, 2)),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(1024, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(2, ),
    ])
    # momentum, adam, sgd, mini-batch. 并非是局部最小点,极大可能是鞍点,高维空间中鞍点数目远大于最优点,但鞍点的数量在整个空间内是微不足道的
    # 真正可能遇到的问题是大面积平坦区域,但是其情况是loss值很高.未知的地形将导致假收敛
    # 控制Learning rate为一较小的量
    # adam = Adam(lr=1e-4), model.compile(optimizer=adam)
    # rmsprop = RMSprop(lr=0.0005, rho=0.9, epsilon=1e-8, decay=0.0); model.compile(optimizer=rmsprop)
nowmodel.compile(optimizer=keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-8), loss='mean_squared_error',
              metrics=['mean_squared_error'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
nowmodel.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 32, 32, 32, 1)     4         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 30, 30, 30, 32)    896       
_________________________________________________________________
average_pooling3d_1 (Average (None, 15, 15, 15, 32)    0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 15, 15, 32)    128       
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 13, 13, 13, 64)    55360     
_________________________________________________________________
average_pooling3d_2 (Average (None, 6, 6, 6, 64)       0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 6, 6, 6, 64)       256       
__________

In [14]:
epochs = 0 

step_epoch = 10

while epochs <= max_epochs:

    print('###############################################')
    print('Begin training for '+modelname+', current epochs = ', epochs, '; max_epochs = ', max_epochs, '...')
    print('###############################################')
    outputf.write('###############################################\n')
    outputf.write('Begin training for '+modelname+', current epochs = '+ str(epochs)+ '; max_epochs = '+str( max_epochs)+ '...\n')
    outputf.write('###############################################\n')

    if True:
        nowmodel.fit_generator(train_generator(),
                    steps_per_epoch=465//batch_size,  # 数据规格可能大小不对应
                    epochs=step_epoch,
                    verbose=1,
                    validation_data=(x_test,y_test))
    epochs += step_epoch

    filepath = './'+modelname+'/'+str(epochs)+'.save'
    # Plot validation
    if True:
        fig, ax = None, None
        for row in range(10):
            x_test, y_test = create_validate_sample(1, use_random=True, startid=row)
            fig, ax = plot_test(nowmodel, x_test, y_test, fig=fig, ax=ax)
        ax.grid(); plt.show()
        ax.set_title('#-epochs = '+str(epochs), fontsize=16)
        fig.savefig(filepath+'.png', format='png')
    print('save model to :', filepath )
    outputf.write('save model to :'+str(filepath)+ '\n')
    keras.models.save_model(nowmodel, filepath)

outputf.close()


###############################################
Begin training for src/model1.1.1_pan/, current epochs =  0 ; max_epochs =  300 ...
###############################################
Epoch 1/10
26/77 [=========>....................] - ETA: 44s - loss: 0.0428 - mean_squared_error: 0.0428

KeyboardInterrupt: 